In [1]:
!pip3 install google-cloud-aiplatform --upgrade --user


In [1]:
from google.cloud import aiplatform
from google.cloud.aiplatform import hyperparameter_tuning as hpt

In [8]:
# The spec of the worker pools including machine type and Docker image
# Be sure to replace IMAGE_URI with the path to your Docker image in GCR
worker_pool_specs = [{
    "machine_spec": {
        "machine_type": "n1-standard-4",
        "accelerator_type": "NVIDIA_TESLA_T4",
        "accelerator_count": 1
    },
    "replica_count": 1,
    "container_spec": {
        "image_uri": "gcr.io/qwiklabs-gcp-01-61f33771c1c9/horse-human:hypertune"   # Replace the {IMAGE_URI} with your Docker image path
    }
}]
# Dicionary representing metrics to optimize.
# The dictionary key is the metric_id, which is reported by your training job,
# And the dictionary value is the optimization goal of the metric.
metric_spec={'accuracy':'maximize'}
# Dictionary representing parameters to optimize.
# The dictionary key is the parameter_id, which is passed into your training
# job as a command line argument,
# And the dictionary value is the parameter specification of the metric.
parameter_spec = {
    "learning_rate": hpt.DoubleParameterSpec(min=0.001, max=1, scale="log"),
    "momentum": hpt.DoubleParameterSpec(min=0, max=1, scale="linear"),
    "num_neurons": hpt.DiscreteParameterSpec(values=[64, 128, 512], scale=None)
}

In [3]:
# Create a new bucket in us-central1 region
!gsutil mb -p qwiklabs-gcp-01-61f33771c1c9 -l us-central1 -b on gs://qwiklabs-gcp-01-61f33771c1c9  # Replace <PROJECT_ID> with your project id
# Copy the content in newly created bucket from the old one
!gsutil cp -r gs://artifacts.qwiklabs-gcp-01-61f33771c1c9.appspot.com/* gs://qwiklabs-gcp-01-61f33771c1c9  # Replace <PROJECT_ID> with your project id

Creating gs://qwiklabs-gcp-01-61f33771c1c9/...
Copying gs://artifacts.qwiklabs-gcp-01-61f33771c1c9.appspot.com/containers/images/sha256:09c222e7ff049ba0c4a9538dc043c93216c670f9f97448f2251b28bd86f54655 [Content-Type=application/octet-stream]...
Copying gs://artifacts.qwiklabs-gcp-01-61f33771c1c9.appspot.com/containers/images/sha256:0c9c6fc70f167fb903c2241a4e3023f475c9ba1f994eac0c2ea621bfc766ebaa [Content-Type=application/octet-stream]...
Copying gs://artifacts.qwiklabs-gcp-01-61f33771c1c9.appspot.com/containers/images/sha256:14d7996407dedf22904703489ffdf6e46e484e7148153902c82743bf5897bce2 [Content-Type=application/octet-stream]...
Copying gs://artifacts.qwiklabs-gcp-01-61f33771c1c9.appspot.com/containers/images/sha256:174d792fb622b52ff6a1754bf5df332cce32b15f5cf667212767e1b6767b2943 [Content-Type=application/octet-stream]...
\ [4 files][  1.0 GiB/  1.0 GiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significan

In [4]:
# Replace (YOUR_BUCKET} with your project ID
my_custom_job = aiplatform.CustomJob(display_name='horses-humans-sdk-job',
                              worker_pool_specs=worker_pool_specs,
                              staging_bucket='gs://qwiklabs-gcp-01-61f33771c1c9')

In [7]:
hp_job = aiplatform.HyperparameterTuningJob(
    display_name='horses-humans-sdk-job',
    custom_job=my_custom_job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    max_trial_count=15,
    parallel_trial_count=3)
hp_job.run()

Creating HyperparameterTuningJob


InvalidArgument: 400 List of found errors:	1.Field: hyperparameter_tuning_job.trial_job_spec.worker_pool_specs[0].container_spec.image_uri; Message: Invalid image URI.	 [field_violations {
  field: "hyperparameter_tuning_job.trial_job_spec.worker_pool_specs[0].container_spec.image_uri"
  description: "Invalid image URI."
}
]